In [25]:
import pandas as pd
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords

tqdm.pandas()

In [26]:
data = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/war_data/training_data/secondary.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54043 entries, 0 to 54042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               54043 non-null  object
 1   comment_text     54043 non-null  object
 2   toxicity         54043 non-null  int64 
 3   severe_toxicity  54043 non-null  int64 
 4   obscene          54043 non-null  int64 
 5   threat           54043 non-null  int64 
 6   insult           54043 non-null  int64 
 7   identity_attack  54043 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 3.3+ MB


In [29]:
processed_data = [remove_stopwords(text).replace(".", "") for text in tqdm(data['comment_text'])]
texts = [[word for word in document.lower().split()] for document in tqdm(processed_data)]


100%|██████████| 54043/54043 [00:00<00:00, 377574.62it/s]


In [30]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in tqdm(texts)]
lda_model = LdaModel(tqdm(corpus), num_topics=10,
                     id2word=dictionary,
                     passes=4, alpha=[0.01]*10,
                     eta=[0.01]*len(dictionary.keys()))
for i, topic in lda_model.show_topics(num_topics=10, formatted=False):
    print('Topic {}: {}'.format(i, ', '.join([word[0] for word in topic])))

100%|██████████| 54043/54043 [00:18<00:00, 2949.18it/s]


Topic 0: russia, ukraine, nato, us, usa, invade, i, it's, america, want
Topic 1: biden, ukraine, war, russia, i, joe, invade, going, oil, putin
Topic 2: a, u, s, n, t, o, war, ukraine, russia, )
Topic 3: russia, invade, biden, ukraine, putin, says, us, ukraine,, china, -
Topic 4: false, flag, taliban, violent, cia, afghanistan, afghan, russian, won, pretext
Topic 5: trump, putin, ukraine, troops, to, nato, he, putin's, president, and
Topic 6: ukraine, us, russia, troops, biden, states, europe, united, invasion, russian
Topic 7: ukraine, us, nato, russia, war, the, government, border, russian, coup
Topic 8: troops, ukraine, border, biden, us, russian, sending, send, sent, american
Topic 9: war, ukraine, us, russia, the, biden, want, this, pushing, start
